In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
handle = '' #인스타그램 아이디
pwd = ''  #인스타그램 비밀번호

In [6]:
keyword= '치즈케이크'   #검색할 키워드
num_of_pages = 800 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐

In [3]:
#함수 정의
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

#URL 가져오기
def InstagramUrlFromKeyword (browser,keyword,num_of_pagedowns):
    keyword_url_encode=quote(keyword)
    url='https://www.instagram.com/explore/tags/'+keyword+'/?hl=ko'
    browser.get(url)
    time.sleep(3)
    login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
    login.click()
    time.sleep(3)
    user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
    pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
    user_id.send_keys(handle)
    pw.send_keys(pwd)
    button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
    button.click()
    time.sleep(3)
    arr_href=[]
    body=browser.find_element_by_tag_name('body')
    for i in range(num_of_pagedowns):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        post=browser.find_elements_by_class_name('v1Nh3')
        for j in post:
            href_str=j.find_element_by_css_selector('a').get_attribute('href')
            arr_href.append(href_str)
    return set(arr_href)    

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

#URL에서 데이터 가져오기
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    
    return datetime, user_name, like_count, tags, location

In [8]:
#크롤링
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
arr=InstagramUrlFromKeyword(browser,keyword,num_of_pages)
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location'})

for url in arr:
        try:
            datetime, user_name, like_count, tags, location = IdHashTagFromInstagram(browser,url)
            insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location}, ignore_index = True)
        except:
            continue

In [112]:
len(arr)

6219

In [79]:
insta_df

,User_Name,Tags,Location,Datetime,Like_Count
0,s2_ds,"[#선물스타그램, #비행기티켓, #선물, #생일선물, #언니, #가족, #애정, #...",선물하는 재미가 쏠쏠,2019년 12월 27일,10
1,changsung.ok,"[#여행, #여행사진, #여행스타그램, #세계여행, #해외여행, #베트남, #베트남...","Đại Nội, Cố Đô Huế",2019년 12월 16일,85
2,eu_uumom_lemont,"[#유유네괌, #여행, #여행스타그램, #취향저격, #오오티디, #ootd, #랩원...",Tumon Bay Beach - Guam,2019년 12월 27일,136
3,changsung.ok,"[#여행, #여행사진, #여행스타그램, #세계여행, #해외여행, #베트남, #베트남...",Quần thể danh thắng Tràng An,2019년 12월 14일,95
4,b2190759_ye,"[#코코넛수용소, #푸꾸옥, #베트남푸꾸옥, #푸꾸옥여행, #푸꾸옥아이델고갈곳, #...","Phu Quoc, Kiến Giang, Vietnam",2019년 12월 27일,3
5,sujin_000,"[#대전, #00, #20, #21, #생애첫, #여권, #해외, #비자, #비행기...",대전서구청,2019년 12월 27일,21
6,_xxixa_,"[#연남동카페, #홍대카페, #패션스타그램, #해외여행, #카페투어, #스타일리스트...",동백포레스트,2019년 12월 27일,12
7,bongbong1228,"[#coffemania, #discovermoscow, #moscow, #russia]",Кофемания / Сoffeemania,2019년 12월 27일,34
8,jo_pft,[],Shanghai Disneyland Park,2019년 12월 26일,39
9,chaeyoung_hi,"[#뉴질랜드워홀, #takapunabeach, #christmas]",Takapuna Beach,2019년 12월 27일,26


In [82]:
#csv 저장
insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='UTF-8')

In [80]:
#빈도분석
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)

c.most_common(50)

[('#해외여행', 35),
 ('#여행', 20),
 ('#여행스타그램', 17),
 ('#여행에미치다', 13),
 ('#travel', 10),
 ('#베트남', 8),
 ('#여행사진', 7),
 ('#vietnam', 7),
 ('#trip', 6),
 ('#세계여행', 6),
 ('#베트남여행', 6),
 ('#travelgram', 5),
 ('#travelstagram', 5),
 ('#landscape', 5),
 ('#world', 5),
 ('#asia', 5),
 ('#viaje', 5),
 ('#viajar', 5),
 ('#voyage', 5),
 ('#일상', 5),
 ('#daily', 5),
 ('#ootd', 4),
 ('#육아', 4),
 ('#괌', 4),
 ('#괌여행', 4),
 ('#neul', 4),
 ('#city', 3),
 ('#unesco', 3),
 ('#worldheritage', 3),
 ('#오오티디', 3),
 ('#guam', 3),
 ('#가족여행', 3),
 ('#소통', 3),
 ('#좋아요', 3),
 ('#좋반', 3),
 ('#팔로우', 3),
 ('#selfie', 3),
 ('#대만', 3),
 ('#대만여행', 3),
 ('#감성', 3),
 ('#힐링', 3),
 ('#nhatrang', 3),
 ('#휴가', 3),
 ('#likeforfollow', 3),
 ('#f4f', 3),
 ('#행복', 2),
 ('#ruin', 2),
 ('#palace', 2),
 ('#historic', 2),
 ('#historicsite', 2)]

In [31]:
df2 = pd.DataFrame(columns = {'word', 'count'})
for word in c:
    df2 = df2.append({'word':word, 'count':c[word]}, ignore_index = True)

In [33]:
df2.to_csv(keyword + '_' + str(num_of_pages) + '_' + 'freq' + '.csv',mode='w', encoding='euc-kr')

In [89]:
keyword= '치즈케이크'   #검색할 키워드
num_of_pages = 800 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐

In [90]:
insta_df = []
df2 = ""
c = ""

In [95]:
arr

{'https://www.instagram.com/p/B6kTJi-FNOB/',
 'https://www.instagram.com/p/B6kLosygPVz/',
 'https://www.instagram.com/p/B6kU8q7g9NX/',
 'https://www.instagram.com/p/B6kPwdmhdnN/',
 'https://www.instagram.com/p/B6iu-IuFB5V/',
 'https://www.instagram.com/p/B6kBsLfHTws/',
 'https://www.instagram.com/p/B6kLIcfgg7T/',
 'https://www.instagram.com/p/B6fewp4jqoZ/',
 'https://www.instagram.com/p/B6kaablF6ue/',
 'https://www.instagram.com/p/B6j_vItAkyb/',
 'https://www.instagram.com/p/B6kMRk3gFJq/',
 'https://www.instagram.com/p/B6kEZaqFZsg/',
 'https://www.instagram.com/p/B6kVFnyAu4G/',
 'https://www.instagram.com/p/B6kIvnvgy0F/',
 'https://www.instagram.com/p/B6kEKotgpuh/',
 'https://www.instagram.com/p/B6kFuW1gAvq/',
 'https://www.instagram.com/p/B6kCHhEH5VB/',
 'https://www.instagram.com/p/B6j_3ccg5ZG/',
 'https://www.instagram.com/p/B6kClsdnB5Q/',
 'https://www.instagram.com/p/B6kCdDQHM7G/',
 'https://www.instagram.com/p/B6kQeTaJ0fD/',
 'https://www.instagram.com/p/B6kVbAUnaOB/',
 'https://

In [63]:
insta_df

,User_Name,Tags,Datetime,Like_Count
0,lovely._.02,[],2019년 12월 27일,28
1,africa_at_home,"[#cebu, #africa, #Cebu, #세부아프리카엣홈, #아프리카엣홈, #딜...",2019년 12월 27일,5
2,heejin______kim,"[#ootd, #f4f, #flf, #책, #책스타그램, #카페, #카페스타그램, ...",2019년 12월 27일,16
3,likechu_,"[#여행에미치다, #여행스타그램, #보라카이, #필리핀, #필리핀여행, #셀스타그램...",2019년 12월 27일,54
4,purebreadgirl,[#껠끄쇼즈],2019년 12월 27일,11
...,...,...,...,...
1426,mysun_complete,"[#도나여행, #뉴질랜드남섬, #Thewaytoparadise, #glenorchy...",2019년 12월 27일,27
1427,home__snaps,"[#소니이미지갤러리, #소니, #소니픽쳐스, #여행, #여행스타그램, #셀카, #d...",2019년 12월 27일,13
1428,yeyta8538,"[#하슬라아트월드, #강릉여행하슬라아트월드짱, #여행에미치다, #여행에미치다, #가...",2019년 12월 27일,3
1429,mamama_0901,"[#크리스마스, #크리스마스선물, #선물스타그램, #선물, #꽃다발, #swarov...",2019년 12월 27일,21


In [116]:
#csv 저장
insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='euc-kr')

UnicodeEncodeError: 'euc_kr' codec can't encode character '\u0e1e' in position 14: illegal multibyte sequence

In [11]:
#csv 저장
#insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='euc-kr')
insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='UTF-8')

In [117]:
#빈도분석
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)

c.most_common(50)

df2 = pd.DataFrame(columns = {'word', 'count'})
for word in c:
    df2 = df2.append({'word':word, 'count':c[word]}, ignore_index = True)


In [87]:
insta_df

,User_Name,Tags,Location,Datetime,Like_Count
0,jeju_dongbaek_kitchen,"[#제주도, #한림맛집, #한림맛집추천, #제주신상맛집, #제주숨은맛집, #애월맛집...",동백키친,2019년 12월 27일,171
1,jujuzzang_,[],순천드라마촬영장,2019년 12월 27일,193
2,hhoneykim,"[#코코리파이프, #대문맛집, #제주, #겨울, #제주여행]",코코리파이프,2019년 12월 27일,2
3,seluv,[],Cafe Re’union_리유니온:재회,2019년 12월 27일,27
4,suaaa_rani,[#naturecanvas],네이처캔버스,2019년 12월 26일,34
5,5joori_,"[#제주, #2일차]",Jeju,2019년 12월 27일,3
6,_charm_y,"[#daily, #일상, #데일리, #일상스타그램, #selfie, #셀카, #셀스...",카멜리아 힐,2019년 12월 27일,13
7,graygrove__,[#에그인헬],그레이그로브,2019년 12월 27일,4
8,near.and,[],Jeju,2019년 12월 27일,392
9,juxzl_,"[#제주, #일상, #데일리, #셀카, #셀피, #셀스타그램, #얼스타그램, #찍스...",에버랜드 (withEverland),2019년 12월 25일,84


In [ ]:
def getLatLng(addr):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
  headers = {"Authorization": "KakaoAK YourAppKey"}
  result = json.loads(str(requests.get(url,headers=headers).text))
  match_first = result['documents'][0]['address']
 
  return float(match_first['y']),float(match_first['x'])

In [9]:
insta_df

,Location,Datetime,Tags,Like_Count,User_Name
0,향리단커피,2019년 12월 22일,"[#향리단스콘, #향남맛집, #스콘, #스콘맛집, #화성카페, #향남카페, #수원카...",48,hrdcoffee
1,Changwon,2019년 12월 30일,[],11,staymood_candle
2,The Cheesecake Factory,2019년 12월 17일,"[#대기20분, #줄줄이소세지, #하와이, #하와이신혼여행, #하와이맛집, #치즈케...",35,_dbddbd_
3,인사이드커피-Insidecoffee,2019년 12월 17일,"[#코흘리게, #못생긴, #나의중학생시절, #기억하는사람, #16년지기, #고니, ...",38,jojosuhyun
4,"Seoul, South Korea",2019년 12월 30일,"[#월요일, #모닝커피, #커피빈, #치즈케이크, #셋이아닌둘이서, #커피스타그램,...",21,eunyoung3280
...,...,...,...,...,...
1098,은계지구,2019년 12월 27일,"[#은계지구, #초밥, #허니콤보, #핫도그, #신전, #오돌뼈, #치즈케이크, #...",41,yang_hj2
1099,얼스어스,2019년 12월 18일,[#얼스어스],21,juny_hoy
1100,스윗레시피 Sweet Recipe,2019년 12월 14일,[#스윗레시피],148,sweet_0310
1101,하우다 과자점,2019년 12월 25일,[],25,hauda_jeju


In [14]:
pip install folium

  Using cached https://files.pythonhosted.org/packages/fd/a0/ccb3094026649cda4acd55bf2c3822bb8c277eb11446d13d384e5be35257/folium-0.10.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/36/1c93318e9653f4e414a2e0c3b98fc898b4970e939afeedeee6075dd3b703/branca-0.3.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


## url 컬럼 추가

In [16]:
#치즈케이크 191230
arr

{'https://www.instagram.com/p/B6XfP5YgO2g/',
 'https://www.instagram.com/p/B6n6jBfFVNK/',
 'https://www.instagram.com/p/B6po6sxghvm/',
 'https://www.instagram.com/p/B6qIMZdprLK/',
 'https://www.instagram.com/p/B6pOCC8jNaf/',
 'https://www.instagram.com/p/B6Cbm_NgITo/',
 'https://www.instagram.com/p/B6b3hF1JD13/',
 'https://www.instagram.com/p/B6rVGXlF8M_/',
 'https://www.instagram.com/p/B6DGuDog8PI/',
 'https://www.instagram.com/p/B6Kdc9gnaLe/',
 'https://www.instagram.com/p/B6ArX0LADcq/',
 'https://www.instagram.com/p/B6LI1M0H67f/',
 'https://www.instagram.com/p/B6rc4Rzgy50/',
 'https://www.instagram.com/p/B6Z6VjnhE6-/',
 'https://www.instagram.com/p/B6KWlPNB0MQ/',
 'https://www.instagram.com/p/B6ccZ0fHYrR/',
 'https://www.instagram.com/p/B6cDzYngbJI/',
 'https://www.instagram.com/p/B6PAt6hFoEM/',
 'https://www.instagram.com/p/B6FAaR9H3Pv/',
 'https://www.instagram.com/p/B6m2YMvJsck/',
 'https://www.instagram.com/p/B6af6UnnHj7/',
 'https://www.instagram.com/p/B6NfYBXlsDc/',
 'https://

In [17]:
insta_df_urladd = insta_df

In [33]:
insta_df_urladd

,Location,Datetime,Tags,Like_Count,User_Name
0,향리단커피,2019년 12월 22일,"[#향리단스콘, #향남맛집, #스콘, #스콘맛집, #화성카페, #향남카페, #수원카...",48,hrdcoffee
1,Changwon,2019년 12월 30일,[],11,staymood_candle
2,The Cheesecake Factory,2019년 12월 17일,"[#대기20분, #줄줄이소세지, #하와이, #하와이신혼여행, #하와이맛집, #치즈케...",35,_dbddbd_
3,인사이드커피-Insidecoffee,2019년 12월 17일,"[#코흘리게, #못생긴, #나의중학생시절, #기억하는사람, #16년지기, #고니, ...",38,jojosuhyun
4,"Seoul, South Korea",2019년 12월 30일,"[#월요일, #모닝커피, #커피빈, #치즈케이크, #셋이아닌둘이서, #커피스타그램,...",21,eunyoung3280
...,...,...,...,...,...
1098,은계지구,2019년 12월 27일,"[#은계지구, #초밥, #허니콤보, #핫도그, #신전, #오돌뼈, #치즈케이크, #...",41,yang_hj2
1099,얼스어스,2019년 12월 18일,[#얼스어스],21,juny_hoy
1100,스윗레시피 Sweet Recipe,2019년 12월 14일,[#스윗레시피],148,sweet_0310
1101,하우다 과자점,2019년 12월 25일,[],25,hauda_jeju


In [36]:
type(insta_df_urladd)

pandas.core.frame.DataFrame

In [37]:
type(arr)

set

In [38]:
insta_df_urladd["URL"] = arr

ValueError: Length of values does not match length of index

In [40]:
pd.merge(insta_df_urladd, arr, left_on='Location', right_index=True)

TypeError: Can only merge Series or DataFrame objects, a <class 'set'> was passed

In [58]:
url

'https://www.instagram.com/p/B6ddE8aJE-J/'

In [64]:
print(p.DataFrame(list(arr)))

NameError: name 'p' is not defined

In [67]:
import pandas as p
p.DataFrame(list(arr))

,0
0,https://www.instagram.com/p/B6XfP5YgO2g/
1,https://www.instagram.com/p/B6n6jBfFVNK/
2,https://www.instagram.com/p/B6po6sxghvm/
3,https://www.instagram.com/p/B6qIMZdprLK/
4,https://www.instagram.com/p/B6pOCC8jNaf/
...,...
6214,https://www.instagram.com/p/B6j0btVH0iV/
6215,https://www.instagram.com/p/B6VS8qHBWmP/
6216,https://www.instagram.com/p/B6rdmlLJKjA/
6217,https://www.instagram.com/p/B6c7YbVhF7e/


In [72]:
urls = p.DataFrame(list(arr))

In [76]:
urls

,0
0,https://www.instagram.com/p/B6XfP5YgO2g/
1,https://www.instagram.com/p/B6n6jBfFVNK/
2,https://www.instagram.com/p/B6po6sxghvm/
3,https://www.instagram.com/p/B6qIMZdprLK/
4,https://www.instagram.com/p/B6pOCC8jNaf/
...,...
6214,https://www.instagram.com/p/B6j0btVH0iV/
6215,https://www.instagram.com/p/B6VS8qHBWmP/
6216,https://www.instagram.com/p/B6rdmlLJKjA/
6217,https://www.instagram.com/p/B6c7YbVhF7e/


In [75]:
insta_df_urladd_new = pd.merge(insta_df_urladd, urls)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [77]:
insta_df_urladd["url"] = urls

In [78]:
insta_df_urladd

,Location,Datetime,Tags,Like_Count,User_Name,url
0,향리단커피,2019년 12월 22일,"[#향리단스콘, #향남맛집, #스콘, #스콘맛집, #화성카페, #향남카페, #수원카...",48,hrdcoffee,https://www.instagram.com/p/B6XfP5YgO2g/
1,Changwon,2019년 12월 30일,[],11,staymood_candle,https://www.instagram.com/p/B6n6jBfFVNK/
2,The Cheesecake Factory,2019년 12월 17일,"[#대기20분, #줄줄이소세지, #하와이, #하와이신혼여행, #하와이맛집, #치즈케...",35,_dbddbd_,https://www.instagram.com/p/B6po6sxghvm/
3,인사이드커피-Insidecoffee,2019년 12월 17일,"[#코흘리게, #못생긴, #나의중학생시절, #기억하는사람, #16년지기, #고니, ...",38,jojosuhyun,https://www.instagram.com/p/B6qIMZdprLK/
4,"Seoul, South Korea",2019년 12월 30일,"[#월요일, #모닝커피, #커피빈, #치즈케이크, #셋이아닌둘이서, #커피스타그램,...",21,eunyoung3280,https://www.instagram.com/p/B6pOCC8jNaf/
...,...,...,...,...,...,...
1098,은계지구,2019년 12월 27일,"[#은계지구, #초밥, #허니콤보, #핫도그, #신전, #오돌뼈, #치즈케이크, #...",41,yang_hj2,https://www.instagram.com/p/B6hlQHuJd-c/
1099,얼스어스,2019년 12월 18일,[#얼스어스],21,juny_hoy,https://www.instagram.com/p/B6KjLwjlwss/
1100,스윗레시피 Sweet Recipe,2019년 12월 14일,[#스윗레시피],148,sweet_0310,https://www.instagram.com/p/B6Igd7AJbZw/
1101,하우다 과자점,2019년 12월 25일,[],25,hauda_jeju,https://www.instagram.com/p/B6aBT51nBFW/


In [79]:
#csv 저장
#insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='euc-kr')
insta_df_urladd.to_csv(keyword + '_' + str(num_of_pages) + '_urladded' + '.csv',mode='w', encoding='UTF-8')

## 데이터 처리

### 위도경도 변환 함수

#### 첫번째 예제

In [113]:
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
    headers = {"Authorization": "KakaoAK 50e9c28d9ce411420f9c7d5d8aa5571e"}
    result = json.loads(str(requests.get(url,headers=headers).text))
    match_first = result['documents'][0]['address']
    
    return float(match_first['y']),float(match_first['x'])

In [114]:
import json
testone = "수원시 장안구"
getLatLng(testone)

(37.30392183215563, 127.01027339958392)

In [115]:
import json
testone = "헬로멜로옐로"
getLatLng(testone)

IndexError: list index out of range

#### 두번째 예제

In [88]:
import requests
import sys
import json

x2=[];d=[];g=[];c=[];dc=[];y2=[]
for i in range(1,len(address["x"])+1):
    url = "https://dapi.kakao.com/v2/local/search/address.json?"
    apikey = "자신의 rest api사용"
    query = address["x"][i]
    r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )
    while True:
        try:
            xx2=r.json()["documents"][0]['address']['x']
            x2.append(xx2)
        except:
            x2.append('NA')
        
        try:
            yy2=r.json()["documents"][0]['address']['y']
            y2.append(yy2)
        except:
            y2.append('NA')
            
        try:
            dd=r.json()["documents"][0]['address']['region_3depth_h_name']
            d.append(dd)
        except:
            d.append('NA')
        try:
            dcdc=r.json()["documents"][0]['address']['h_code']
            dc.append(dcdc)
        except:
            dc.append('NA')
        try:
            gg=r.json()["documents"][0]['address']['region_2depth_name']
            g.append(gg)
        except:
            g.append('NA')
        try:
            cc=r.json()["documents"][0]['address']['region_1depth_name']
            c.append(cc)
        except:
            c.append('NA')
        
        break

Address={'Longitude':x2,'Latitude':y2,'H_name':d,'G_name':g,'C_name':c}
Address=pd.DataFrame(Address)
Address.to_csv('KR_AGNCINFO_address.csv', sep=",",encoding='ms949') 

NameError: name 'address' is not defined

#### 세번째 예제

In [ ]:
#카카오맵으로 올리브영 주소 가져오기
#단점: 내 위치기반 500개까지만 보여주더라...ㅎ
    
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
driver=webdriver.Chrome(executable_path=r'C:\Users\user\Documents\Python Scripts\chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://map.kakao.com/')
driver.find_element_by_css_selector('.img_slide.btn_close').click()
driver.find_element_by_css_selector('.layer_body').click()
driver.find_element_by_css_selector('#header input[type="text"]').send_keys('올리브영')
driver.find_element_by_id('search.keyword.submit').click()
driver.execute_script("document.body.style.zoom='80%'")
driver.implicitly_wait(3)
driver.execute_script("window.scrollTo(0, 100)") 
#driver.find_element_by_partial_link_text("아현역점").send_keys(Keys.ENTER)
element = driver.find_element_by_id('info.search.place.more')
driver.execute_script("arguments[0].click();", element)
time.sleep(5)
addr1=[];addr2=[];name=[]    
#1페이지
html = driver.page_source
soup = bs(html, 'html.parser')
for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
    if i %2!=0: #도로명 주소
        a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
        addr2.append(a)
        
    if i %2==0: #지번 주소
        a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
        addr1.append(a)
    
    #매장이름
for i in range(len(soup.select('.PlaceItem.clickArea .link_name'))):
    n=soup.select('.PlaceItem.clickArea .link_name')[i].text
    name.append(n)
while True:
    #2페이지
    elem=driver.find_element_by_id("info.search.page.no2")
    driver.execute_script("arguments[0].click();", elem)
    time.sleep(5)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
        if i %2!=0: #도로명 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr2.append(a)
        if i %2==0: #지번 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr1.append(a)
        #매장이름
    for i in range(len(soup1.select('.PlaceItem.clickArea .link_name'))):
        n=soup.select('.PlaceItem.clickArea .link_name')[i].text
        name.append(n)
    #3페이지
    elem=driver.find_element_by_id("info.search.page.no3")
    driver.execute_script("arguments[0].click();", elem)
    time.sleep(8)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
        if i %2!=0: #도로명 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr2.append(a)
        if i %2==0: #지번 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr1.append(a)
        #매장이름
    for i in range(len(soup1.select('.PlaceItem.clickArea .link_name'))):
        n=soup.select('.PlaceItem.clickArea .link_name')[i].text
        name.append(n)
    #4페이지
    elem=driver.find_element_by_id("info.search.page.no4")
    driver.execute_script("arguments[0].click();", elem)
    time.sleep(7)
    #html = driver.execute_script("return arguments[0].innerHTML;", elem)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
        if i %2!=0: #도로명 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr2.append(a)
        if i %2==0: #지번 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr1.append(a)
        #매장이름
    for i in range(len(soup1.select('.PlaceItem.clickArea .link_name'))):
        n=soup.select('.PlaceItem.clickArea .link_name')[i].text
        name.append(n)
    #5페이지
    elem=driver.find_element_by_id("info.search.page.no5")
    driver.execute_script("arguments[0].click();", elem)
    time.sleep(6)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
        if i %2!=0: #도로명 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr2.append(a)
        if i %2==0: #지번 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr1.append(a)
        #매장이름
    for i in range(len(soup1.select('.PlaceItem.clickArea .link_name'))):
        n=soup.select('.PlaceItem.clickArea .link_name')[i].text
        name.append(n) 
    element=driver.find_element_by_id('info.search.page.next')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(7)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    for i in range(len(soup.select('.PlaceItem.clickArea .addr > p'))):
        if i %2!=0: #도로명 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr2.append(a)
        if i %2==0: #지번 주소
            a=soup.select('.PlaceItem.clickArea .addr > p')[i].text
            addr1.append(a)
        #매장이름
    for i in range(len(soup1.select('.PlaceItem.clickArea .link_name'))):
        n=soup.select('.PlaceItem.clickArea .link_name')[i].text
        name.append(n) 

### 크롤링으로 위도,경도 수집

In [ ]:
#URL에서 데이터 가져오기
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    
    return datetime, user_name, like_count, tags, location


#크롤링
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
arr=InstagramUrlFromKeyword(browser,keyword,num_of_pages)
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location'})

for url in arr:
        try:
            datetime, user_name, like_count, tags, location = IdHashTagFromInstagram(browser,url)
            insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location}, ignore_index = True)
        except:
            continue

### 지도 URL 수집 크롤링 -작동함---- 임시용

In [ ]:
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
def get_location_url(url):
    browser.get(url)
    try:
        loc_url = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').get_attribute('href')
    except:
        loc_url = 'N/A'
    return loc_url

insta_df['Location_URL'] = insta_df['URL'].apply(get_location_url)

insta_df.to_csv('rawdata_mapurladded.csv',mode='w', encoding='UTF-8')

In [5]:
insta_df = pd.read_csv('rawdata.csv', index_col=0, encoding='UTF-8')

### 지도 URL에서 Latitude, Longitude 수집 크롤링

In [6]:
def get_coord(url):
    browser.get(url)
    
    lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
    long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    return (lat, long)

#로그인 부분
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle)
pw.send_keys(pwd)
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)



insta_df['Latitude'] = insta_df['Location_URL'].apply(get_coord[0])
insta_df['Longitude'] = insta_df['Location_URL'].apply(get_coord[1])

TypeError: 'function' object is not subscriptable

In [8]:
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle)
pw.send_keys(pwd)
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)



for url in insta_df['Location_URL']:
    browser.get(url)
    lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
    long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
#    insta_df = insta_df.append(pd.DataFrame([[url,lat,long]], columns=['Latitude','Longitude']),ignore_index=True)
    insta_df.append({'Latitude':lat, 'Longitude':long}, ignore_index=True)
#    insta_df['Longitude'] = long
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//meta[@property='place:location:latitude']"}
  (Session info: chrome=79.0.3945.88)


In [18]:
# 로그인
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle)
pw.send_keys(pwd)
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)

#주소URL에서 위도, 경도 가져오기
def grab_and_clean_selenium(url):
    browser.get(url)
    try:
        lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
        long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    except:
        lat = 'N/A'
        long= 'N/A'
    return lat,long

for url in test_df['Location_URL']:
    lat, long = grab_and_clean_selenium(url)
#    insta_df = insta_df.append(pd.DataFrame([[url,lat,long]], columns=['Latitude','Longitude']),ignore_index=True)
#    insta_df['Longitude'] = long
#    insta_df.append({'Latitude':lat, 'Longitude':long}, ignore_index=True)
    print(lat, long)



35.1796 129.076
35.158281645912 129.0646983492
37.567941879641 126.82650544431
37.51502 127.01648
37.588577270508 127.08880615234
37.477829 126.888667
35.661523635094 139.69790457651
37.565528 126.923266
35.156224641722 129.06466627239
35.234465142745 128.87947395456


In [16]:
test_df.to_csv('rawdata_coordadded_test.csv', mode='w', encoding='UTF-8')

In [11]:
insta_df.to_csv('rawdata_coordadded.csv',mode='w', encoding='UTF-8')

In [14]:
test_df = insta_df[:10]

In [47]:
# 로그인
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle)
pw.send_keys(pwd)
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)

#주소URL에서 위도, 경도 가져오기
def grab_and_clean_selenium(url):
    browser.get(url)
    try:
        lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
        long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    except:
        lat = 'N/A'
        long= 'N/A'
    return lat,long

coord = pd.DataFrame(columns=['Latitude','Longitude','Location_URL'])
for url in test_df['Location_URL']:
    lat, long = grab_and_clean_selenium(url)
    coord = coord.append({'Latitude':lat,'Longitude':long,'Location_URL':url}, ignore_index=True)
#    insta_df = insta_df.append(pd.DataFrame([[url,lat,long]], columns=['Latitude','Longitude']),ignore_index=True)
#    insta_df['Longitude'] = long
#    insta_df.append({'Latitude':lat, 'Longitude':long}, ignore_index=True)



In [39]:
coord

,Latitude,Longitude,Location_URL
0,35.1796,129.076,https://www.instagram.com/explore/locations/28...
1,35.158281645912,129.0646983492,https://www.instagram.com/explore/locations/28...
2,37.567941879641,126.82650544431,https://www.instagram.com/explore/locations/34...
3,37.51502,127.01648,https://www.instagram.com/explore/locations/21...
4,37.588577270508,127.08880615234,https://www.instagram.com/explore/locations/11...
5,37.477829,126.888667,https://www.instagram.com/explore/locations/29...
6,35.661523635094,139.69790457651,https://www.instagram.com/explore/locations/10...
7,37.565528,126.923266,https://www.instagram.com/explore/locations/87...
8,35.156224641722,129.06466627239,https://www.instagram.com/explore/locations/39...
9,35.234465142745,128.87947395456,https://www.instagram.com/explore/locations/18...


In [40]:
test_df

,Datetime,Tags,Like_Count,Location,User_Name,URL,Location_URL
0,2019년 12월 22일,"['#부산', '#서울', '#친구', '#밥', '#스타벅스', '#케이크', '...",25,"Busan, South Korea",_ye._.yomi_,https://www.instagram.com/p/B6W9LyMhTjB/,https://www.instagram.com/explore/locations/28...
1,2019년 12월 29일,"['#카페', '#카페그램', '#카페투어', '#카페추천', '#신상카페', '#...",56,향미공간:香味空間,flavor_space,https://www.instagram.com/p/B6pC7XtJ-s7/,https://www.instagram.com/explore/locations/28...
2,2019년 12월 22일,"['#익스큐즈미', '#익스큐즈미카페', '#카페투어', '#카페', '#카페스타그...",33,마곡지구,borami_ing,https://www.instagram.com/p/B6V3bf5gQyF/,https://www.instagram.com/explore/locations/34...
3,2019년 12월 19일,"['#아인슈페너', '#연유카페라떼', '#치즈케이크', '#우리동네카페', '#까...",13,"Seoul, Korea",revi_ewfood,https://www.instagram.com/p/B6PRZImh242/,https://www.instagram.com/explore/locations/21...
4,2019년 12월 29일,"['#디저트', '#치즈케이크', '#바닐라', '#뜨아', '#더벤티']",7,The venti 면목점,kyong_jin_life_record,https://www.instagram.com/p/B6n1h6Fpnrx/,https://www.instagram.com/explore/locations/11...
5,2019년 12월 23일,"['#홍대', '#홍대카페', '#먹스타그램', '#디저트그램', '#치즈케이크']",21,C27 cheesecake&coffee,88.10.21,https://www.instagram.com/p/B6aoUK4BpQV/,https://www.instagram.com/explore/locations/29...
6,2019년 12월 29일,"['#shibuya', '#치즈케이크', '#일본']",87,CCC Cheese Cheers Cafe Shibuya,non1101.___,https://www.instagram.com/p/B6p6mZLnxUg/,https://www.instagram.com/explore/locations/10...
7,2019년 12월 29일,['#어나더룸'],96,Another Room,snjlee_,https://www.instagram.com/p/B6pxwzonI0p/,https://www.instagram.com/explore/locations/87...
8,2019년 12월 29일,[],58,"Goodgoodweather,굿굿웨더",goodgoodweather,https://www.instagram.com/p/B6pEirIJhJS/,https://www.instagram.com/explore/locations/39...
9,2019년 12월 27일,"['#스템스커피', '#stemscoffee', '#듁스커피', '#dukescof...",43,Stemscoffee,stemscoffee,https://www.instagram.com/p/B6j2lCfhZNC/,https://www.instagram.com/explore/locations/18...


In [41]:
pd.merge(test_df,coord)

,Datetime,Tags,Like_Count,Location,User_Name,URL,Location_URL,Latitude,Longitude
0,2019년 12월 22일,"['#부산', '#서울', '#친구', '#밥', '#스타벅스', '#케이크', '...",25,"Busan, South Korea",_ye._.yomi_,https://www.instagram.com/p/B6W9LyMhTjB/,https://www.instagram.com/explore/locations/28...,35.1796,129.076
1,2019년 12월 29일,"['#카페', '#카페그램', '#카페투어', '#카페추천', '#신상카페', '#...",56,향미공간:香味空間,flavor_space,https://www.instagram.com/p/B6pC7XtJ-s7/,https://www.instagram.com/explore/locations/28...,35.158281645912,129.0646983492
2,2019년 12월 22일,"['#익스큐즈미', '#익스큐즈미카페', '#카페투어', '#카페', '#카페스타그...",33,마곡지구,borami_ing,https://www.instagram.com/p/B6V3bf5gQyF/,https://www.instagram.com/explore/locations/34...,37.567941879641,126.82650544431
3,2019년 12월 19일,"['#아인슈페너', '#연유카페라떼', '#치즈케이크', '#우리동네카페', '#까...",13,"Seoul, Korea",revi_ewfood,https://www.instagram.com/p/B6PRZImh242/,https://www.instagram.com/explore/locations/21...,37.51502,127.01648
4,2019년 12월 29일,"['#디저트', '#치즈케이크', '#바닐라', '#뜨아', '#더벤티']",7,The venti 면목점,kyong_jin_life_record,https://www.instagram.com/p/B6n1h6Fpnrx/,https://www.instagram.com/explore/locations/11...,37.588577270508,127.08880615234
5,2019년 12월 23일,"['#홍대', '#홍대카페', '#먹스타그램', '#디저트그램', '#치즈케이크']",21,C27 cheesecake&coffee,88.10.21,https://www.instagram.com/p/B6aoUK4BpQV/,https://www.instagram.com/explore/locations/29...,37.477829,126.888667
6,2019년 12월 29일,"['#shibuya', '#치즈케이크', '#일본']",87,CCC Cheese Cheers Cafe Shibuya,non1101.___,https://www.instagram.com/p/B6p6mZLnxUg/,https://www.instagram.com/explore/locations/10...,35.661523635094,139.69790457651
7,2019년 12월 29일,['#어나더룸'],96,Another Room,snjlee_,https://www.instagram.com/p/B6pxwzonI0p/,https://www.instagram.com/explore/locations/87...,37.565528,126.923266
8,2019년 12월 29일,[],58,"Goodgoodweather,굿굿웨더",goodgoodweather,https://www.instagram.com/p/B6pEirIJhJS/,https://www.instagram.com/explore/locations/39...,35.156224641722,129.06466627239
9,2019년 12월 27일,"['#스템스커피', '#stemscoffee', '#듁스커피', '#dukescof...",43,Stemscoffee,stemscoffee,https://www.instagram.com/p/B6j2lCfhZNC/,https://www.instagram.com/explore/locations/18...,35.234465142745,128.87947395456


In [46]:
#첫번째크롤링후
coord

,Latitude,Longitude,Location_URL
0,35.1796,129.076,https://www.instagram.com/explore/locations/28...
1,35.158281645912,129.0646983492,https://www.instagram.com/explore/locations/28...
2,37.567941879641,126.82650544431,https://www.instagram.com/explore/locations/34...
3,37.51502,127.01648,https://www.instagram.com/explore/locations/21...
4,37.588577270508,127.08880615234,https://www.instagram.com/explore/locations/11...
5,37.477829,126.888667,https://www.instagram.com/explore/locations/29...
6,35.661523635094,139.69790457651,https://www.instagram.com/explore/locations/10...
7,37.565528,126.923266,https://www.instagram.com/explore/locations/87...
8,35.156224641722,129.06466627239,https://www.instagram.com/explore/locations/39...
9,35.234465142745,128.87947395456,https://www.instagram.com/explore/locations/18...


In [48]:
#두번째크롤링후, 행이 중복되진 않는지 - 않네 초기화되네
coord

,Latitude,Longitude,Location_URL
0,35.1796,129.076,https://www.instagram.com/explore/locations/28...
1,35.158281645912,129.0646983492,https://www.instagram.com/explore/locations/28...
2,37.567941879641,126.82650544431,https://www.instagram.com/explore/locations/34...
3,37.51502,127.01648,https://www.instagram.com/explore/locations/21...
4,37.588577270508,127.08880615234,https://www.instagram.com/explore/locations/11...
5,37.477829,126.888667,https://www.instagram.com/explore/locations/29...
6,35.661523635094,139.69790457651,https://www.instagram.com/explore/locations/10...
7,37.565528,126.923266,https://www.instagram.com/explore/locations/87...
8,35.156224641722,129.06466627239,https://www.instagram.com/explore/locations/39...
9,35.234465142745,128.87947395456,https://www.instagram.com/explore/locations/18...


### 지도 URL에서 Latitude, Longitude 수집 크롤링_____최종

In [56]:
#raw data 로드
insta_df = pd.read_csv('testtesttest1.csv', index_col=0, encoding='UTF-8')

# 로그인
browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle) #아이디
pw.send_keys(pwd) #비밀번호
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)

#주소URL에서 위도, 경도 가져오기
def grab_and_clean_selenium(url):
    browser.get(url)
    try:
        lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
        long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    except:
        lat = 'N/A'
        long= 'N/A'
    return lat,long

coord = pd.DataFrame(columns=['Latitude','Longitude','Location_URL'])
for url in insta_df['Location_URL']:
    lat, long = grab_and_clean_selenium(url)
    coord = coord.append({'Latitude':lat,'Longitude':long,'Location_URL':url}, ignore_index=True)

insta_df_final = pd.merge(insta_df,coord)
insta_df_final.to_csv('rawdata_final.csv',mode='w', encoding='UTF-8')

KeyboardInterrupt: 